In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import json
import sys

directory = "data_11-whitespace-aug"
user_data = directory #  + "/train"
valid_data = directory #  + "/val"
test_data = directory   + "/label_book" # this can be the label book, or any other test set you create
# test_data = directory + "/test" # this can be the label book, or any other test set you create

### DO NOT MODIFY BELOW THIS LINE, THIS IS THE FIXED MODEL ###
batch_size = 8
tf.random.set_seed(123)


if __name__ == "__main__":
    train = tf.keras.preprocessing.image_dataset_from_directory(
        user_data + '/train',
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=True,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
    )

    valid = tf.keras.preprocessing.image_dataset_from_directory(
        user_data + '/val',
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=True,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
    )

    total_length = ((train.cardinality() + valid.cardinality()) * batch_size).numpy()
    if total_length > 10_000:
        print(f"Dataset size larger than 10,000. Got {total_length} examples")
        sys.exit()

    test = tf.keras.preprocessing.image_dataset_from_directory(
        test_data,
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=False,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
    )

    base_model = tf.keras.applications.ResNet50(
        input_shape=(32, 32, 3),
        include_top=False,
        weights=None,
    )
    base_model = tf.keras.Model(
        base_model.inputs, outputs=[base_model.get_layer("conv2_block3_out").output]
    )

    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = tf.keras.applications.resnet.preprocess_input(inputs)
    x = base_model(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(10)(x)
    model = tf.keras.Model(inputs, x)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=0.0001),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    model.summary()
    loss_0, acc_0 = model.evaluate(valid, verbose=0)
    print(f"loss {loss_0}, acc {acc_0}")

    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        "best_model",
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        train,
        validation_data=valid,
        epochs=10,
        callbacks=[checkpoint],
        verbose=2
    )

   

Found 5587 files belonging to 10 classes.
Found 1559 files belonging to 10 classes.
Found 52 files belonging to 10 classes.
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 tf.__operators__.getitem (S  (None, 32, 32, 3)        0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 32, 32, 3)        0         
                                                                 
 model (Functional)          (None, 8, 8, 256)         229760    
                                                                 
 global_average_pooling2d (G  (None, 256)              0         
 lobalAveragePooling2D)                                          
 

C:\Users\anjali\anaconda3\envs\dvc\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


loss 34.016395568847656, acc 0.09236690402030945
Epoch 1/10
699/699 - 32s - loss: 1.6668 - accuracy: 0.4285 - val_loss: 1.2214 - val_accuracy: 0.5581 - 32s/epoch - 46ms/step
Epoch 2/10
699/699 - 23s - loss: 1.0139 - accuracy: 0.6603 - val_loss: 0.8212 - val_accuracy: 0.7460 - 23s/epoch - 33ms/step
Epoch 3/10
699/699 - 23s - loss: 0.7491 - accuracy: 0.7582 - val_loss: 0.7591 - val_accuracy: 0.7396 - 23s/epoch - 33ms/step
Epoch 4/10
699/699 - 24s - loss: 0.5870 - accuracy: 0.8099 - val_loss: 0.5536 - val_accuracy: 0.8185 - 24s/epoch - 34ms/step
Epoch 5/10
699/699 - 23s - loss: 0.4707 - accuracy: 0.8527 - val_loss: 0.6253 - val_accuracy: 0.7858 - 23s/epoch - 33ms/step
Epoch 6/10
699/699 - 23s - loss: 0.4020 - accuracy: 0.8701 - val_loss: 0.4708 - val_accuracy: 0.8563 - 23s/epoch - 33ms/step
Epoch 7/10
699/699 - 23s - loss: 0.3270 - accuracy: 0.9023 - val_loss: 0.4587 - val_accuracy: 0.8473 - 23s/epoch - 33ms/step
Epoch 8/10
699/699 - 23s - loss: 0.3001 - accuracy: 0.9071 - val_loss: 0.411

In [5]:
 model.load_weights("best_model")

loss, acc = model.evaluate(valid, verbose=0)
print(f"final loss {loss}, final acc {acc}")

test_loss, test_acc = model.evaluate(test, verbose=0)
print(f"test loss {test_loss}, test acc {test_acc}")


final loss 0.4111422896385193, final acc 0.8723540902137756
test loss 1.2760112285614014, test acc 0.7307692170143127


In [10]:
jsonString = json.dumps({"train_loss":loss , "train_acc": acc}, indent=4)
jsonFile = open("test_result.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

In [11]:
jsonString = json.dumps({"train_loss":test_loss , "test_acc": test_acc}, indent=4)
jsonFile = open("train_result.json", "w")
jsonFile.write(jsonString)
jsonFile.close()